<div style="text-align: left;" role="presentation">
<h1 role="presentation"><strong><span role="presentation">Lecci&oacute;n: 3 Miércoles 04 de Marzo</span></strong></h1>

# Operaciones Geometricas:

* Sea $A\in\mathbb{R}^{m\times n}$ la representacion en double de una imagen. Sea $A(x,y)$ la entrada $(x,y)$ de la imagen $A$. Deseamos modificar el orden de las entradas $(x,y)$ de $A$ para obtener una nueva imagen $B$ tal que.

$$A(x,y) = B(x',y')$$
    

* El cambiar la entrada $A(x,y)$ a la posicion $(x',y')$ se realiza a partir de una transformacion.

$$(x',y') = T(x,y) = (T_{x}(x,y), T_{y}(x,y))$$

donde  $T$ es la transformacion ccompuesta por 2 subtransformaciones $T_{x}$, $T_{y}$.
    
* Nosotros Veremos un conjunto de transformaciones lineales(llamdas afines) de la forma.

$$X' = Tx(x,y)=a_{0}x+a_{1}y+a_{2}$$
$$Y' = Ty(x,y)=b_{0}x+b_{1}y+b_{2}$$

* Las ecuaciones anteriores pueden ser representadas como:

$$\begin{bmatrix}X'\\Y'\\1\end{bmatrix} = \begin{bmatrix}a_{0}& a_{1} & a_{2}\\b_{0}& b_{1} & b_{2}\\0 & 0 & 1\end{bmatrix}\begin{bmatrix}x\\ 1\\1\end{bmatrix}$$

<h3><center>Tabla de Transformadas en Octave</center></h3>

|Transformada|$a{0}$|$a{1}$|$a{2}$|$b{0}$|$b{1}$|$b{2}$|
|---|---|---|---|---|---|---|
|$$Translacion \Delta{x},\Delta{y}$$ | $$1$$ | $$0$$ | $$\Delta{x} $$ | $$0$$ | $$1$$ | $$\Delta{y}$$|
|$$Escalar [S_{x},S_{x}]$$ | $$S_{x}$$ | $$0$$ | $$0$$ | $$0$$ | $$S_{y}$$ | $$0$$|
|$$Rotar \theta$$ | $$cos(\theta)$$ | $$sin(\theta)$$ | $$0$$ | $$-sin(\theta)$$ | $$cos(\theta)$$ | $$0$$ |

In [3]:
function Y=rotar(A, angulo)
  #A es la matriz que representa la imagen (8 bits)
  #Angulo debe estar en grados
  #Y es la imagen rotada
  a0=cosd(angulo); a1=sind(angulo); a2=0;
  b0=-sind(angulo); b1=cosd(angulo); b2=0;
  
  %Centro
  [m,n,r] = size(A);
  xc=round(m/2); yc=round(n/2);
  Y=zeros(m,n,r);
  Y=uint8(Y);
  
  for x=1:m
    for y=1:n
      xnew= mod(round(a0*(x-xc)+a1*(y-yc)+xc),m);
      ynew= mod(round(b0*(x-xc)+b1*(y-yc)+yc),n);
      xaux= round(a0*(x-xc)+a1*(y-yc)+xc);
      yaux= round(b0*(x-xc)+b1*(y-yc)+yc);
      if and(xnew==xaux, ynew==yaux)
        Y(xnew+1, ynew+1, :) = A(x,y,:);
      endif
    endfor
  endfor
  
  
endfunction

In [ ]:
clc; clear;
pkg load image; %Carga el paquete de imagenes

A = imread('files/barbara.jpg');

%Rotar de la imagen A
deltax = 200; deltay = 150;

angulo = 30;
a0=cosd(angulo); a1=sind(angulo); a2=0;
b0=-sind(angulo); b1=cosd(angulo); b2=0;

T = [a0 a1 a2; b0 b1 b2; 0 0 1];

Tr = maketform('affine', T'); %Crear la transformacion

B=rotar(A, 40);
C=promedio(B)

subplot(1,3,1) 
imshow(A)
title('Normal')

subplot(1,3,2)
imshow(B)
title('rotar(A, 40)')

subplot(1,3,3)
imshow(C)
title('rotar(A, 40) + funcion promedio')

# Interpolacion:

Asuma la siguiente parte de una imagen

<table style="width:25%">
  <tr>
    <td>(x-1,y-1)</td>
    <td>(x-1,y)</td>
    <td>(x-1,y+1)</td>
  </tr>
  <tr>
    <td>(x,y-1)</td>
    <td>(x,y)</td>
    <td>(x,y+1)</td>
  </tr>
  <tr>
    <td>(x+1,y-1)</td>
    <td>(x+1,y)</td>
    <td>(x+1,y+1)</td>
  </tr>
</table>

El método de interpolación consiste en aproximar el valor de un pixel de una imagen, utilizando la información de los pixeles que hay al rededor. Un método de interpolacion es el de promedio.


In [1]:
function B = promedio(A)
  #A es la matriz a la cual de le hara limpieza
  [m,n,r] = size(A);
  A = im2double(A);
  B = A;
  for x=1:m
    for y=1:n
     if or((A(x,y,1) == 0), (A(x,y,2) == 0), (A(x,y,3) == 0))
       if and(x==1, y==1)
        newpixel = (A(x,y+1,:)+A(x+1,y,:)+A(x+1,y+1,:))/3;
       elseif and(x==m,y==1)
         newpixel = (A(x-1,y,:)+A(x-1,y+1,:)+A(x,y+1,:))/3;
       elseif and(x==1,y==n)
         newpixel = (A(x,y-1,:)+A(x+1,y-1,:)+A(x+1,y,:))/3;
       elseif and(x==m,y==n)
         newpixel = (A(x-1,y,:)+A(x-1,y-1,:)+A(x,y-1,:))/3;
       elseif and(x>1,y>1,x<m,y<n)
         newpixel = (A(x-1,y-1,:)+A(x,y-1,:)+A(x+1,y-1,:)+A(x+1,y,:)+A(x+1,y+1,:)+A(x,y+1,:)+A(x-1,y+1,:)+A(x-1,y,:))/8;
       elseif y==1
         newpixel = (A(x-1,y,:)+A(x-1,y+1,:)+A(x,y+1,:)+A(x+1,y+1,:)+A(x+1,y,:))/5;
       elseif x==1
         newpixel = (A(x,y-1,:)+A(x+1,y-1,:)+A(x+1,y,:)+A(x+1,y+1,:)+A(x,y+1,:))/5;
       elseif x==m
         newpixel = (A(x,y-1,:)+A(x-1,y-1,:)+A(x-1,y,:)+A(x-1,y+1,:)+A(x,y+1,:))/5;
       elseif y==n
         newpixel = (A(x-1,y,:)+A(x-1,y-1,:)+A(x,y-1,:)+A(x+1,y-1,:)+A(x+1,y,:))/5;
       endif
       B(x,y,:) = newpixel(1,1,:);
     else
       continue
     endif
    endfor
  endfor
endfunction